In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import random
from selenium.webdriver.chrome.options import Options

In [46]:
class Naver_Cafe_Crawling():
    def __init__(self):    #크롬 창을 연다
        
        path = 'C:/informs/chromedriver'
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument('--disable-java')
#         chrome_options.add_argument("--headless")   # headless 
        
        profile = {"plugins.plugins_disabled": ["Shockwave Flash"]}
        chrome_options.add_experimental_option("prefs", profile)
        self.browser = webdriver.Chrome(executable_path= path, chrome_options=chrome_options)

#         executable_path=r
    
    def login_naver(self):
        #naver에 로그인 한다
        url_login = 'https://nid.naver.com/nidlogin.login'
        self.browser.get(url_login)
        print('네이버 로그인 페이지에 접근합니다')

        #네이버 아이디/패스워드가져오기
        with open('c:/informs/naver.txt','r') as idpw:
            naver_id, naver_pw = idpw.readline().split(',')

        #네이버 아이디/로그인 입력하기
        e = self.browser.find_element_by_id('id')
        e.clear()   #기존에 입력된 값 삭제
        e.send_keys(naver_id)
        e = self.browser.find_element_by_id('pw')
        e.clear()
        e.send_keys(naver_pw)
        form = self.browser.find_element_by_css_selector('input.btn_global[type=submit]')
        form.submit()
        
    def enter_naver_cafe(self, cafe_url):
        # 까페에 접속한다
        self.browser.get(cafe_url)
        print('네이버까페에 접속합니다')
        print('\n!!!!!!!!!!! 플래시 설정을 점검해주세요 !!!!!!!!!!')

    def get_menu(self,cafe_url):
        '''
        네이버 까페 url을 입력 --> 까페 clubid / 메뉴그룹/메뉴명/메뉴번호를 리턴한다.
        '''
        self.enter_naver_cafe(cafe_url)
        
        response = self.browser.page_source
        html=BeautifulSoup(response, 'lxml')
        
        menu_groups=html.select('ul.cafe-menu-list')        #메뉴 그룹 리스트
        menu_group_names=html.select('div.cafe-menu-tit')  #메뉴그룹명 리스트     
           
        for menu_group, group_name in zip(menu_groups, menu_group_names):      #메뉴 그룹별로 조회함
            menus=menu_group.select('li a')
            for menu in menus:
                menu_name=menu.text            #메뉴 명
                menu_num=menu['id'][8:]        #메뉴(카테고리) 번호
                menu_url=urljoin(cafe_url, menu['href'])     #메뉴 주소 가져오기(상대경로 --> 절대경로)
                print("  - {} [{}] : {}".format(menu_name, menu_num,menu_url))
            print("{}".format(group_name.text.strip()))   #메뉴그룹명을 가져옴
            
        
    def searching_word(self, word):
        # 까페 내 검색어로 검색 --> 검색결과 페이지
        e = self.browser.find_element_by_id('topLayerQueryInput')
        e.clear()
        e.send_keys(word)
        form = self.browser.find_element_by_css_selector('#cafe-search > form > a')
        form.click()
        print('{}에 대한 결과를 검색합니다'.format(word))
        self.switch_frame()
        
    def switch_frame(self):
        try: 
            self.browser.switch_to_frame('cafe_main') 
            print('  (프레임 변경했습니다)')
        except: None
        
        
    def move_next_pages(self, i):
        #i page로 이동함
        #1번페이지부터 순차적으로 이동 필요함(한 페이지내  1.2.3.....10. 다음 /   이전.11.12.13.....20.다음 으로 구성되어있음)
        if i<=11: k=i
        else:
            k = (i-2)%10 + 3
        
        try: 
            next = self.browser.find_element_by_xpath('//*[@id="main-area"]/div/table/tbody/tr/td[{}]/a'.format(k))
            next.click()
        except: return True        # 다음페이지가 없다면 True 를 리턴한다

    
    
    def get_content(self):

        date = self.browser.find_element_by_css_selector('td.m-tcol-c.date').text        #작성일
        title = self.browser.find_element_by_css_selector('span.b.m-tcol-c').text        #글 제목
        writer = self.browser.find_element_by_css_selector('td.p-nick > a.m-tcol-c.b').text  #작성자
        content = self.browser.find_element_by_id('tbody').text.strip().replace('\n','')  #본문내용
        board = self.browser.find_element_by_css_selector('tr > td > a.m-tcol-c').text   #게시판명
        
        reply_writers = self.browser.find_elements_by_css_selector('a.m-tcol-c._rosRestrict._nickUI')
        reply_dates = self.browser.find_elements_by_css_selector('span.date.m-tcol-c.filter-50')
        reply_contents = self.browser.find_elements_by_css_selector('p.comm.m-tcol-c > span.comm_body')
        
        
        replys = []
        for i in range(len(reply_writers)):
            replys.append((i+1,reply_writers[i].text,reply_dates[i].text,reply_contents[i].text))
                
        reply_counts = len(replys)
        
        bottom = self.browser.find_element_by_css_selector('#cmtMenu > div.fl.reply_sort > table > tbody > tr').text
        bottom = bottom.split(' ')
        read_counts = bottom[bottom.index('조회수')+1]
        likes_counts = bottom[bottom.index('좋아요')+1]

            
        
        results ={
            'title' : title,
            'writer' : writer,
            'date' : date,
            'content' : content,
            'board' : board,
            'read_counts' : read_counts,
            'likes_counts' : likes_counts,
            'reply_counts' : reply_counts,
            'replys' : replys
        }
        
        print('  - 크롤링중: {} '.format(title))
        return results

    
    def back_to_searching(self):
        self.browser.find_element_by_css_selector('#main-area > div.list-btn-nor2.upper-list > div.fr > div.btn2').click()
    
    
            
    def get_contents_in_page(self):
            
        contents = []
        articles = self.browser.find_elements_by_css_selector('span > span.aaa > a.m-tcol-c')
        
        for i in range(len(articles)):
            articles = self.browser.find_elements_by_css_selector('span > span.aaa > a.m-tcol-c')
            article = articles[i]
            article.click()
            content = self.get_content()
            contents.append(content)
            time.sleep(random.uniform(0.2,0.7))  #추가시간 대기
            self.back_to_searching()            
            
        return contents
    



In [108]:
# 테스트

In [54]:
# naver = Naver_Cafe_Crawling()
# naver.login_naver()   #네이버에 로그인한다

cafe_url = 'http://cafe.naver.com/mp3musicdownloadcafe'   #화장발
# cafe_url = 'http://cafe.naver.com/cosmania'
naver.enter_naver_cafe(cafe_url)   #접속 후 플래시 On 해줘야 함
# naver.get_menu(cafe_url)

네이버까페에 접속합니다

!!!!!!!!!!! 플래시 설정을 점검해주세요 !!!!!!!!!!


In [55]:
word = '모노아이즈'

# naver.login_naver()   #네이버에 로그인한다
# naver.enter_naver_cafe(cafe_url)   #자바스크립트 오픈 해줘야함
naver.searching_word(word)

naver.switch_frame()

results = []

for i in range(10,12):   # 시작페이지는 1로 지정
    print('<진행중> {}페이지 수집중 - 현재 수집된 글 {}'.format(i,len(results)))
    check = naver.move_next_pages(i)
    
    if check:
        break
    
    time.sleep(1+random.uniform(0.2,0.5))
    results+=naver.get_contents_in_page()

    

print('<종료> {}에 대한 글 {}개를 수집하였습니다. '.format(word, len(results)))
# naver.browser.quit()



모노아이즈에 대한 결과를 검색합니다
  (프레임 변경했습니다)
<진행중> 10페이지 수집중 - 현재 수집된 글 0
  - 크롤링중: 아리따움 모노아이즈 샀어요 또 샀어요.. 
  - 크롤링중: [아리따움] 아리따움 모노아이즈 추천 , 아리따움 모노아이즈 비교, 아리따움 모노아이즈 트러플, 진저파우더, 얼쓰 발색 비교 
  - 크롤링중: [아리따움] 모노아이즈 세일 1+1 구매 후기 [메탈릭 브라운 / 이브닝] 
  - 크롤링중: [아리따움] 모노아이즈 킬링레드 , 오키드 메이크업 ♪ 
  - 크롤링중: 에뛰드하우스 룩 앳 마이 아이즈 쥬얼 오픈금지 판도라 / 퍼플메이크업 ♪ 
  - 크롤링중: 데일리 오렌지 메이크업 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.33.506120 (e3e53437346286c0bc2d2dc9aa4915ba81d9023f),platform=Windows NT 6.3.9600 x86_64)


In [49]:
def get_contents_in_page(self):

    contents = []
    articles = self.browser.find_elements_by_css_selector('span > span.aaa > a.m-tcol-c')
    print(articles[0])
    print(articles[1])
    
    articles[1].click()
    print('1')
    naver.back_to_searching()
    articles[2].click()
    print('2')
    
    
    
    
#     for i in range(len(articles)):
# #             selector = '//*[@id="main-area"]/div/form/table/tbody/tr/td/span/span/a[]'.format(i+1)
# #             selector = '//*[@id="main-area"]/div/form/table/tbody/tr[{}]/td/span/span/a'.format(2*i+1)
# #                         //*[@id="main-area"]/div[7]/form/table/tbody/tr[4]/td[2]/span/span/a
#         article = articles[i]
#         print(article)
# #             article = self.browser.find_element_by_xpath(selector)
#         time.sleep(random.uniform(0.2,0.7))
#         naver.switch_frame()
#         article.click()
        
#         content = self.get_content()
#         contents.append(content)
#         time.sleep(random.uniform(2.1,2.8))  #추가시간 대기
#         self.back_to_searching()            

#     return contents

get_contents_in_page(naver)

IndexError: list index out of range

In [161]:
a = naver.browser.find_element_by_css_selector('#cmtMenu > div.fl.reply_sort > table > tbody > tr').text
b = a.split(' ')
b[b.index('좋아요')+1]
# b.index('좋아요')

'0'

In [16]:
print(len(results))

17


In [9]:
print(results[0])

{'title': '촉촉보송한 벨벳틴트 추천 :: [네이크업페이스] 벨벳 스캔들 틴트 4호 원나잇스캔들 & 5호 란제리스캔들', 'writer': '여시캣(celi****)', 'date': '2017.11.24. 17:38', 'content': "(썸네일)립덕후가 오늘도 립제품을 추천하러 왔습니당>3<예전에 기존 3컬러 모두 써보고 반했었던 제품인데이번에 신규 컬러가 2가지 추가되었더라구요출시 일주일만에 전색상이 완판된 바 있는SNS 화제의 품절대란 틴트,네이크업페이스 벨벳 스캔들 !신규 컬러 2종 원나잇스캔들 & 란제리스캔들함께 보고 가시죵~네이크업페이스 벨벳 스캔들은지모추출물이 함유되어입술에 가볍고 부드럽게 녹아드는벨벳 볼륨을 연출해주는 벨벳틴트에요~참고로 지모는 백합과의 풀인데립 플럼퍼, 탄력 화장품 등에 많이 쓰이는'보르피린'이라는 성분이 바로요 지모에서 나온 추출물이더라구요+_+그런 지모추출물이 들어있는 만큼바르면 정말 예쁜 입술 볼륨을 표현해준답니다벨벳스캔들의 새로운 컬러4호 원나잇 스캔들과 5호 란제리 스캔들입니다이름이 둘다 야시꾸리함ㅋㅋㅋㅋㅋ기존에 있던누드스캔들, 웨딩스캔들, 키스스캔들은mlbb 말린장미 계열이었는데이번 컬러는 맑은 레드와 여리한 핑크군요!!둘다 제가 넘 좋아하는 계열의 컬러라발라보기도 전에 두근....사각사각한 디자인이 참 예뻐요붉은 뚜껑은 매트한 메탈 느낌이고투명한 부분조차 무광이라서 더욱 세련되어 보이죵!벨벳스캔들 틴트 뚜껑을 열어어플리케이터를 촵 꺼내봅니다끝쪽이 날렵한 세모꼴의 팁이라바를 때 진짜 편하더라구요~입술산 깔끔하게 연출하기도 좋고입꼬리처럼 좁은 부위를세밀하게 채워주기에도 좋아요'3'벨벳스캔들 4호 원나잇스캔들 & 5호 란제리스캔들팔뚝 발색 및 착색입니다꺅 둘이 뭔가 되게 상반되는? 느낌의 컬러인데둘다 진짜 이쁘지 않나요ㅋㅋㅋ핰...텍스처는 촉촉하고 부드러운 느낌의크림 같은 제형으로,넘나 매끈하고 촉촉하게 발려요~한번만 슥 그어도불투명하고 진하게 발색이 잘나오구요보시다시피 착색력도 좋은 편이에요!저거 착색 진짜

In [17]:
# json 으로 저장하기
from datetime import datetime
import json

now = datetime.now()
INPUT_FILE = './data_crwal/화장발_모노아이즈_2p_{}{}{}{}.txt'.format(now.month,now.day,now.hour,now.minute)
with open(INPUT_FILE,'w', encoding = 'utf-8') as save:
    json.dump(results, save, ensure_ascii=False)    # 한글 표시를 위해 ensure_ascii = False 옵션을 넣어줌

In [ ]:
#json 불러오기
results = []
INPUT_FILE = './data_crwal/화장발_모노아이즈_2p_1128160.txt'
with open(INPUT_FILE, 'r', encoding ='utf-8') as save:
    results = json.load(save)
print(len(results))
print(results[0])